In [31]:

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, GPT2Tokenizer
from fancy_einsum import einsum

In [32]:
model_name = "meta-llama/Llama-3.1-8B"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [33]:
token_embeds = model.model.embed_tokens.weight
value_vectors = torch.cat(
    [
        model.model.layers[layer_idx].mlp.down_proj.weight
        for layer_idx in range(model.config.num_hidden_layers)
    ],
    dim=0,
)

In [34]:
seed_token_toxic = ["fuck", "shit", "crap"]

toxic_token_id = []
for tok in seed_token_toxic:
    ids = tokenizer(tok, add_special_tokens=False)["input_ids"]
    if len(ids) == 1:
        toxic_token_id.append(ids[0])
    else:
        print(f"Skipping '{tok}' → {ids} (not a single token)")
print(toxic_token_id)

[71574, 41153, 99821]


In [35]:
toxic_embed = token_embeds[toxic_token_id].mean(dim=0)

In [21]:
# seed_token_toxic = ["fuck", "cunt", "shit", "crap", "asshole", "bullshit"]

# toxic_token_id = []
# for tok in seed_token_toxic:
#     ids = tokenizer(tok, add_special_tokens=False)["input_ids"]
#     if len(ids) == 1:
#         toxic_token_id.append(ids[0])
#     else:
#         print(f"Skipping '{tok}' → {ids} (not a single token)")


Skipping 'cunt' → [66, 3935] (not a single token)
Skipping 'asshole' → [395, 31520] (not a single token)
Skipping 'bullshit' → [43104, 41153] (not a single token)


In [27]:
def unembed_to_text(vector, model, tokenizer, k=10):
    # LayerNorm before output projection 
    norm = model.model.norm
    lm_head = model.lm_head.weight 

    # Normalize the input vector using final layer norm
    vector = norm(vector)

    # Compute similarity scores with vocab embeddings
    dots = einsum("vocab d, d -> vocab", lm_head, vector)
    top_k = dots.topk(k).indices

    return tokenizer.batch_decode(top_k, skip_special_tokens=True)

In [36]:
k = 20
target_vec = toxic_embed

# Compute raw dot products between value vectors and the toxic direction
dot_prods = torch.einsum("nd,d->n", value_vectors, target_vec)

# Top-k most aligned MLP value vectors
top_value_vecs = dot_prods.topk(k).indices

# Print top vectors with layer and index info
for vec_idx in top_value_vecs:
    layer = vec_idx.item() // model.config.intermediate_size
    idx_in_layer = vec_idx.item() % model.config.intermediate_size
    print(f"Value vec: Layer {layer}, index {idx_in_layer}")
    print(unembed_to_text(value_vectors[vec_idx], model, tokenizer))


RuntimeError: einsum(): subscript d has size 4096 for operand 1 which does not broadcast with previously seen size 14336